In [1]:
from optimal_sizing_library import *

In [2]:
DATE_OF_SIMULATION = '2021-04-02'
SIGMA_SL = -1
SIGMA_TP = 1.5
MAX_TRADE_DURATION = 63
PTF_VOL_TGT = 0.15
AUM_LOST_SL = 0.02

In [3]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

In [7]:
mc_fcast_ptf_volatility(df,
                                    DATE_OF_SIMULATION,
                                    SIGMA_SL,
                                    SIGMA_TP,
                                    MAX_TRADE_DURATION, 
                                    34,
                                    AUM_LOST_SL,
                                    iterations=100)

100%|██████████| 200/200 [02:02<00:00,  1.63it/s]

Estimated Portfolio Volatility: 13.600000000000001% ± 1.77%
